In [1]:
def k_1_0(S1, S2, D1, D2):
    return -2*D2 + 2*S2

def k_0_1(S1, S2, D1, D2):
    return -2*D1 + L

def k_2_0(S1, S2, D1, D2):
    return 2*D2*(D2 - 1) - 4*D2*S2 + 2*(S2 -1)*S2

def k_1_1(S1, S2, D1, D2):
    return 2*(1 + 2*D1 -L)*(D2 - S2)

def k_0_2(S1, S2, D1, D2):
    return 2*D1**2 - 2*L*D1 + .5*(L-1)*L

### Additive + Dominant

In [4]:
from gpytorch.constraints import Positive
from gpytorch.constraints import LessThan
from EpiK.kernels import Kernel

class K11(Kernel):
    """Diploid kernel"""

    is_stationary = True

    # We will register the parameter when initializing the kernel
    def __init__(self, 
                par_prior=None, par_constraint=None, 
                **kwargs):
      super().__init__(**kwargs)

      # register the raw parameter
      self.register_parameter(
          name='raw_par', 
          parameter=torch.nn.Parameter(torch.zeros(*self.batch_shape, 2))
      )

      # set the parameter constraint to be positive, when nothing is specified
      if par_constraint is None:
          par_constraint = LessThan(upper_bound=0.)

      # register the constraint
      self.register_constraint("raw_par", par_constraint)


    # now set up the 'actual' paramter
    @property
    def par(self):
      # when accessing the parameter, apply the constraint transform
      return self.raw_par_constraint.transform(self.raw_par)
    @par.setter
    def par(self, value):
      return self._set_par(value)


    def forward(self, geno1, geno2, **params):

        global L
        L = geno1.shape[1]
        
        geno1_ht = 1.*(geno1 == 1.)
        geno2_ht = 1.*(geno2 == 1.)        
        geno1_h0 = 1.*(geno1 == 0.)
        geno1_h1 = 1.*(geno1 == 2.)
        geno2_h0 = 1.*(geno2 == 0.)
        geno2_h1 = 1.*(geno2 == 2.)

        S1 = self.covar_dist(geno1_ht, geno2_ht, **params)
        S2 = self.covar_dist(geno1_h0, geno2_h0, **params) + self.covar_dist(geno1_h1, geno2_h1, **params)
        D2 = self.covar_dist(geno1_h0, geno2_h1, **params) + self.covar_dist(geno1_h1, geno2_h0, **params)
        D1 = L - S1 - S2 - D2
        
        


        return torch.exp(self.par[0])*k_1_0(S1, S2, D1, D2) + torch.exp(self.par[1])*k_0_1(S1, S2, D1, D2)


In [5]:
# define kernel

ker = K11()

NameError: name 'torch' is not defined

In [103]:
# test forward

ker(train_x, train_x).evaluate()

tensor([[ 2.0955, -0.0917,  0.0195,  ...,  0.0859, -0.1426,  0.1265],
        [-0.0917,  2.2501, -0.0166,  ..., -0.0120,  0.0301,  0.3722],
        [ 0.0195, -0.0166,  2.1351,  ...,  0.2024,  0.0504, -0.1873],
        ...,
        [ 0.0859, -0.0120,  0.2024,  ...,  1.9307,  0.5068,  0.1488],
        [-0.1426,  0.0301,  0.0504,  ...,  0.5068,  1.9660,  0.0907],
        [ 0.1265,  0.3722, -0.1873,  ...,  0.1488,  0.0907,  2.0311]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [98]:
# train model
print("training GP model using CV")
ker, likelihood = train_model_cv(ker, train_x, train_y, 50, .1)

training GP model using CV
working on iteration 0
working on iteration 10
working on iteration 20
working on iteration 30
working on iteration 40


In [99]:
# make predictions - build model
torch.cuda.empty_cache()
model = models.ExactGPModel(train_x, train_y, likelihood, ker).to(output_device)

In [100]:
partition_size = partition_sizes.partitions[i]

model.eval()
likelihood.eval()
with gpytorch.beta_features.checkpoint_kernel(train_x.shape[0]//int(partition_size)):
    f_preds = model(test_x)

f_mean = f_preds.mean.cpu().detach().numpy()
y_test = test_y.detach().cpu().numpy()
r2_score = r2(y_test, f_mean)                
print(r2_score)

0.3508992254412563


### Pairwise Epistatic

In [158]:
from gpytorch.constraints import Positive
from gpytorch.constraints import LessThan
from EpiK.kernels import Kernel

class K20(Kernel):
    """Diploid kernel"""

    is_stationary = True

    # We will register the parameter when initializing the kernel
    def __init__(self, 
                par_prior=None, par_constraint=None, 
                **kwargs):
      super().__init__(**kwargs)

      # register the raw parameter
      self.register_parameter(
          name='raw_par', 
          parameter=torch.nn.Parameter(torch.zeros(*self.batch_shape, 2))
      )

      # set the parameter constraint to be positive, when nothing is specified
      if par_constraint is None:
          par_constraint = LessThan(upper_bound=0.)

      # register the constraint
      self.register_constraint("raw_par", par_constraint)


    # now set up the 'actual' paramter
    @property
    def par(self):
      # when accessing the parameter, apply the constraint transform
      return self.raw_par_constraint.transform(self.raw_par)
    @par.setter
    def par(self, value):
      return self._set_par(value)


    def forward(self, geno1, geno2, **params):

        global L
        L = geno1.shape[1]
        
        geno1_ht = 1.*(geno1 == 1.)
        geno2_ht = 1.*(geno2 == 1.)        
        geno1_h0 = 1.*(geno1 == 0.)
        geno1_h1 = 1.*(geno1 == 2.)
        geno2_h0 = 1.*(geno2 == 0.)
        geno2_h1 = 1.*(geno2 == 2.)

        S1 = self.covar_dist(geno1_ht, geno2_ht, **params)
        S2 = self.covar_dist(geno1_h0, geno2_h0, **params) + self.covar_dist(geno1_h1, geno2_h1, **params)
        D2 = self.covar_dist(geno1_h0, geno2_h1, **params) + self.covar_dist(geno1_h1, geno2_h0, **params)
        D1 = L - S1 - S2 - D2
        
        


        return torch.exp(self.par[0])*k_1_0(S1, S2, D1, D2) + torch.exp(self.par[1])*k_2_0(S1, S2, D1, D2)


In [159]:
# define kernel

ker = K20()

In [160]:
ker.raw_par = torch.nn.Parameter(torch.tensor([0., -torch.log(torch.tensor(10000.))]))

In [171]:
ker.par

tensor([ -9.1755, -17.2649], device='cuda:0', grad_fn=<AddBackward0>)

In [167]:
# test forward

ker(train_x, train_x).evaluate()

tensor([[ 8.7927e+01,  5.4029e-01,  9.3856e-02,  ...,  2.2749e-01,
         -3.2287e+00,  3.2500e+00],
        [ 5.4029e-01,  1.0533e+02, -2.3651e-01,  ...,  3.8844e+00,
          5.0192e+00,  1.5172e+01],
        [ 9.3856e-02, -2.3651e-01,  9.2249e+01,  ...,  3.3238e+00,
          2.7738e+00, -3.1521e+00],
        ...,
        [ 2.2749e-01,  3.8844e+00,  3.3238e+00,  ...,  7.0900e+01,
          1.5270e+01,  7.8569e+00],
        [-3.2287e+00,  5.0192e+00,  2.7738e+00,  ...,  1.5270e+01,
          7.4418e+01,  2.6287e+00],
        [ 3.2500e+00,  1.5172e+01, -3.1521e+00,  ...,  7.8569e+00,
          2.6287e+00,  8.1081e+01]], device='cuda:0', grad_fn=<AddBackward0>)

In [172]:
# train model
print("training GP model using CV")
ker, likelihood = train_model_cv(ker, train_x, train_y, 50, .1)

training GP model using CV
working on iteration 0
working on iteration 10
working on iteration 20
working on iteration 30
working on iteration 40


In [173]:
# make predictions - build model
torch.cuda.empty_cache()
model = models.ExactGPModel(train_x, train_y, likelihood, ker).to(output_device)

In [174]:
partition_size = partition_sizes.partitions[i]

model.eval()
likelihood.eval()
with gpytorch.beta_features.checkpoint_kernel(train_x.shape[0]//int(partition_size)):
    f_preds = model(test_x)

f_mean = f_preds.mean.cpu().detach().numpy()
y_test = test_y.detach().cpu().numpy()
r2_score = r2(y_test, f_mean)                
print(r2_score)

0.3502157336240562


### Pairwise Epistatic + Dominant

In [205]:
from gpytorch.constraints import Positive
from gpytorch.constraints import LessThan
from EpiK.kernels import Kernel

class K2(Kernel):
    """Diploid kernel"""

    is_stationary = True

    # We will register the parameter when initializing the kernel
    def __init__(self, 
                par_prior=None, par_constraint=None, 
                **kwargs):
      super().__init__(**kwargs)

      # register the raw parameter
      self.register_parameter(
          name='raw_par', 
          parameter=torch.nn.Parameter(torch.zeros(*self.batch_shape, 5))
      )

      # set the parameter constraint to be positive, when nothing is specified
      if par_constraint is None:
          par_constraint = LessThan(upper_bound=0.)

      # register the constraint
      self.register_constraint("raw_par", par_constraint)


    # now set up the 'actual' paramter
    @property
    def par(self):
      # when accessing the parameter, apply the constraint transform
      return self.raw_par_constraint.transform(self.raw_par)
    @par.setter
    def par(self, value):
      return self._set_par(value)


    def forward(self, geno1, geno2, **params):

        global L
        L = geno1.shape[1]
        
        geno1_ht = 1.*(geno1 == 1.)
        geno2_ht = 1.*(geno2 == 1.)        
        geno1_h0 = 1.*(geno1 == 0.)
        geno1_h1 = 1.*(geno1 == 2.)
        geno2_h0 = 1.*(geno2 == 0.)
        geno2_h1 = 1.*(geno2 == 2.)

        S1 = self.covar_dist(geno1_ht, geno2_ht, **params)
        S2 = self.covar_dist(geno1_h0, geno2_h0, **params) + self.covar_dist(geno1_h1, geno2_h1, **params)
        D2 = self.covar_dist(geno1_h0, geno2_h1, **params) + self.covar_dist(geno1_h1, geno2_h0, **params)
        D1 = L - S1 - S2 - D2
        
        Ks = torch.stack([k_1_0(S1, S2, D1, D2), k_0_1(S1, S2, D1, D2), k_1_1(S1, S2, D1, D2), 
                    k_2_0(S1, S2, D1, D2), k_0_2(S1, S2, D1, D2)])
        Ks_reweighted = torch.mul(torch.exp(self.par).unsqueeze(1).unsqueeze(1), Ks)


        return torch.sum(Ks_reweighted, dim=0)
    


In [241]:
# define kernel

ker = K2()

In [262]:
orders = [1, 1, 200, 200, 200]

log_par0 = -2*torch.log(torch.tensor(orders))

In [263]:
ker.raw_par = torch.nn.Parameter(log_par0)

In [264]:
ker = ker.to(output_device)

In [265]:
# test forward

ker(train_x, train_x).evaluate()

tensor([[11381.7090,  -397.0153,    82.1189,  ...,   371.4546,  -531.2731,
           495.4877],
        [ -397.0153, 12386.7285,   -67.0020,  ...,  -122.4593,    38.8508,
          1452.1964],
        [   82.1189,   -67.0020, 11636.0859,  ...,   841.4955,   166.1476,
          -712.8038],
        ...,
        [  371.4546,  -122.4593,   841.4955,  ..., 10343.4844,  2125.5254,
           516.4810],
        [ -531.2731,    38.8508,   166.1476,  ...,  2125.5254, 10563.2266,
           346.6054],
        [  495.4877,  1452.1964,  -712.8038,  ...,   516.4810,   346.6054,
         10971.6328]], device='cuda:0', grad_fn=<SumBackward1>)

In [272]:
# train model
print("training GP model using CV")
ker, likelihood = train_model_cv(ker, train_x, train_y, 50, .1)

training GP model using CV
working on iteration 0
working on iteration 10
working on iteration 20
working on iteration 30
working on iteration 40


In [273]:
# make predictions - build model
torch.cuda.empty_cache()
model = models.ExactGPModel(train_x, train_y, likelihood, ker).to(output_device)

In [274]:
partition_size = partition_sizes.partitions[i]

model.eval()
likelihood.eval()
with gpytorch.beta_features.checkpoint_kernel(train_x.shape[0]//int(partition_size)):
    f_preds = model(test_x)

f_mean = f_preds.mean.cpu().detach().numpy()
y_test = test_y.detach().cpu().numpy()
r2_score = r2(y_test, f_mean)                
print(r2_score)

0.37757590030985866


In [275]:
ker.par

tensor([ -0.9894,  -2.4193, -11.1024,  -9.1453,  -9.3189], device='cuda:0',
       grad_fn=<AddBackward0>)

### All epistatic

### All dominant

### Low order + All order

In [7]:
import pandas as pd
import os
import gc
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import GPUtil

import torch
import gpytorch

from scipy.stats import pearsonr
from scipy.special import binom as binom
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split

In [8]:
import sys
sys.path.insert(1, '../')
import EpiK.models as models

In [9]:
output_device = 0
n_devices = torch.cuda.device_count()
models.set_params(output_device, n_devices)
print("number of GPUs = {}; output device = {}".
      format(n_devices, torch.cuda.current_device()))

number of GPUs = 8; output device = 0


In [10]:
from EpiK.functions import get_data, get_envs, set_data_path
set_data_path("../matsui_data/")

In [11]:
# training sizes
props = [.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95]

In [12]:
# check_point sizes
partitions = [2, 2, 2, 2, 4, 4, 4, 4, 16, 32, 64, 120]
pd.DataFrame({"props":props, "partitions":partitions}).to_csv("partition_sizes.csv", index=None)

In [13]:
partition_sizes = pd.read_csv("partition_sizes.csv")

### Data

In [14]:
env_list = get_envs()
env = env_list[5]

In [15]:
geno_t, pheno = get_data(env)

/blue/juannanzhou/EpiK/notebooks/../EpiK/functions.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  geno_t = torch.tensor(geno_t, dtype=torch.float)


In [16]:
inds_sub = np.where(np.array(pheno.pheno < -0.6) == False)[0]

In [17]:
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% | 45% |
|  1 |  0% |  0% |
|  2 |  0% |  0% |
|  3 |  0% |  0% |
|  4 |  0% |  0% |
|  5 |  0% |  0% |
|  6 |  0% |  0% |
|  7 |  0% |  0% |


In [31]:
def get_train_test(sub, sub_t):
    train_x = geno_t[sub]
    train_y = torch.tensor(np.array(pheno.pheno[sub]), dtype=torch.float32)
    test_x = geno_t[sub_t]
    test_y = torch.tensor(np.array(pheno.pheno[sub_t]), dtype=torch.float32)
    train_x, train_y = train_x.contiguous(), train_y.contiguous()
    test_x, test_y = test_x.contiguous(), test_y.contiguous()
    train_x, train_y = train_x.to(output_device), train_y.to(output_device)
    test_x, test_y = test_x.to(output_device), test_y.to(output_device)
    return train_x, test_x, train_y, test_y

In [32]:
i=0 
np.random.seed(100)
train_size = np.round(props[i]*len(inds_sub)).astype('int')
sub = np.random.choice(inds_sub, train_size)
sub_t = np.random.choice(list(set(inds_sub).difference(sub)), 2000)

In [33]:
train_x, test_x, train_y, test_y = get_train_test(sub, sub_t)

### Train model

In [36]:
import EpiK.functions
from EpiK.functions import train_model_cv

EpiK.functions.output_device = output_device
EpiK.functions.n_devices = n_devices

In [46]:
# define kernel

from EpiK.kernels import DiKernel
ker = DiKernel()
ker.raw_lda = torch.nn.Parameter(torch.tensor(-8.))
ker.raw_eta = torch.nn.Parameter(torch.tensor(-12.))

In [55]:
# define kernel

from EpiK.kernels import RBFKernel
ker = RBFKernel()

EpiK.kernels.L = geno_t.shape[1]
# ls = torch.tensor([[96.04]]).to(output_device)
# ker = gpytorch.kernels.RBFKernel().to(output_device)

# ker.lengthscale = ls

In [59]:
# define kernel

from EpiK.kernels import LinKernel
ker = LinKernel()

In [ ]:
# train model
print("training GP model using CV")
ker, likelihood = train_model_cv(ker, train_x, train_y, 50, .1)

In [61]:
# make predictions - build model
torch.cuda.empty_cache()
model = models.ExactGPModel(train_x, train_y, likelihood, ker).to(output_device)

In [62]:
partition_size = partition_sizes.partitions[i]

model.eval()
likelihood.eval()
with gpytorch.beta_features.checkpoint_kernel(train_x.shape[0]//int(partition_size)):
    f_preds = model(test_x)

f_mean = f_preds.mean.cpu().detach().numpy()
y_test = test_y.detach().cpu().numpy()
r2_score = r2(y_test, f_mean)                
print(r2_score)

/home/juannanzhou/.local/lib/python3.9/site-packages/gpytorch/utils/linear_cg.py:321: NumericalWarning: CG terminated in 1000 iterations with average residual norm 2176170.0 which is larger than the tolerance of 0.01 specified by gpytorch.settings.cg_tolerance. If performance is affected, consider raising the maximum number of CG iterations by running code in a gpytorch.settings.max_cg_iterations(value) context.
  warnings.warn(


-4023169547572.15


In [64]:
def k_1_0(S1, S2, D1, D2):
    return -2*D2 + 2*S2

def k_0_1(S1, S2, D1, D2):
    return -2*D1 + L

def k_2_0(S1, S2, D1, D2):
    return 2*(d2 - 1) - 4*d2*s2 + 2*(s2 -1)*s2

def k_1_1(S1, S2, D1, D2):
    return 2*(1 + 2*d1 -L)*(D2 - S2)

def k_0_2(S1, S2, D1, D2):
    return 2*d1**2 - 2*L*d1 + .5*(L-1)*L

In [5]:
geno_np = geno_t.cpu().numpy()

NameError: name 'geno_t' is not defined

In [4]:
import pandas as pd
geno_pd = pd.DataFrame(geno_np)

NameError: name 'geno_np' is not defined